In [11]:
from data.loader import HAM10000Dataset
from data.transforms import get_transform
from models.adaptive_activation import AdaptiveActivation, ParametricActivation
from models.autoencoder import Autoencoder
import torch
import torch.optim as optim
import os
from torch import nn
from utils.training import load_checkpoint



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

activation = torch.load('pre_act/activation.pth')

root_path = 'outputs'
encoder_name = 'autoencoder'
model = Autoencoder(input_channels=3, activation=activation, 
                    parametric_activation=True, device=device).to(device)
optimizer = optim.Adam(model.parameters())
checkpoint_filename = os.path.join(root_path, 'checkpoints', f'{encoder_name}_checkpoint.pth')
start_epoch, saved_loss = load_checkpoint(model, optimizer, checkpoint_filename)



C:\Users\The_Boss\AppData\Local\Temp\ipykernel_16332\2884339221.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  activation = torch.load('pre_act/activation.pth')


Early stopping at epoch 21386 - No improvement for 100 epochs
Checkpoint loaded from outputs\checkpoints\autoencoder_checkpoint.pth


d:\Learnable_Activation\utils\training.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename)


In [12]:
from utils.training import load_checkpoint
from models.autoencoder import Autoencoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

activation = torch.load('pre_act/activation.pth')

root_path = 'outputs'
encoder_name = 'autoencoder'
model = Autoencoder(input_channels=3, activation=activation, 
                    parametric_activation=True, device=device).to(device)
optimizer = optim.Adam(model.parameters())
checkpoint_filename = os.path.join(root_path, 'checkpoints', f'{encoder_name}_checkpoint.pth')
start_epoch, saved_loss = load_checkpoint(model, optimizer, checkpoint_filename)

C:\Users\The_Boss\AppData\Local\Temp\ipykernel_16332\1066767035.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  activation = torch.load('pre_act/activation.pth')


Early stopping at epoch 18279 - No improvement for 100 epochs
Checkpoint loaded from outputs\checkpoints\autoencoder_checkpoint.pth


In [13]:
# Init dataloaders
transform = get_transform(train=True)
ham10000_dataset_train = HAM10000Dataset(root_path='D:/Learnable_Activation/datasets/HAM10000', transform=transform, split='train')
ham10000_dataset_test = HAM10000Dataset(root_path='D:/Learnable_Activation/datasets/HAM10000', transform=transform, split='test')

ham10000_loader_train = torch.utils.data.DataLoader(ham10000_dataset_train, batch_size=32, shuffle=True)
ham10000_loader_test = torch.utils.data.DataLoader(ham10000_dataset_test, batch_size=32, shuffle=True)

In [14]:
class Classifier(nn.Module):
    def __init__(self, encoders, num_classes, device):
        super(Classifier, self).__init__()
        self.encoders = encoders
        
        # Convolutional layers for initial feature extraction
        self.conv1 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Transformer Encoder
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=64, nhead=4), num_layers=2
        )
        
        # Classification layers
        self.fc1 = nn.Linear(64, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)
        
        self.device = device

    def forward(self, x):
        # Get encoded features and average them
        encoded = [encoder(x) for encoder in self.encoders]
        encoded = torch.stack(encoded).mean(dim=0)
        
        # Process through conv layers
        x = self.conv1(encoded)
        x = self.bn1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = nn.ReLU()(x)
        
        # Global average pooling
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        
        # Prepare for transformer
        x = x.unsqueeze(1)  # Add sequence dimension
        x = self.transformer_encoder(x)  # Pass through transformer
        
        # Classification
        x = x.mean(dim=1)  # Average over the sequence
        x = self.fc1(x)
        x = self.dropout(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        
        return x

In [15]:
# Create the classifier with 7 classes from HAM10000
classifier = Classifier(encoders=model.encoders, num_classes=7, device=device).to(device)

c:\Users\The_Boss\.conda\envs\UNI_GPT\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [17]:
criterion = nn.CrossEntropyLoss()
classifier_optimizer = optim.Adam(classifier.parameters())

def train_epoch(classifier, train_loader, optimizer, criterion):
    classifier.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = classifier(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss

def evaluate(classifier, test_loader):
    classifier.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            output = classifier(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)
    return correct / total

# Training loop
num_epochs = 100
best_accuracy = 0.0

for epoch in range(num_epochs):
    total_loss = train_epoch(classifier, ham10000_loader_train, classifier_optimizer, criterion)
    accuracy = evaluate(classifier, ham10000_loader_test)
    
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Total Loss: {total_loss:.4f}')
    print(f'Accuracy: {accuracy:.4f}')
    
    if accuracy > best_accuracy:
        print('Saving best model...')
        best_accuracy = accuracy
        torch.save(classifier.state_dict(), 'best_classifier.pth')
    
    print('-' * 50)

print(f'Best Accuracy: {best_accuracy:.4f}')

Epoch [1/100]
Total Loss: 283.7147
Accuracy: 0.6680
Saving best model...
--------------------------------------------------


KeyboardInterrupt: 